# Strands Agents サンプル

このノートブックでは、Strands Agentsの基本的な使い方から高度な機能まで、段階的に学習できます。

## 目次
1. 環境設定とインポート
2. 基本的な使用例（シンプルな質問応答）
3. MCPツール統合
4. ストリーミング応答
5. デバッグとエラーハンドリング

## 1. 環境設定とインポート

In [1]:
# 必要なライブラリのインポート
import os
import logging
from dotenv import load_dotenv
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters

# ログレベルの設定
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# 環境変数の読み込み
load_dotenv()

print("✅ ライブラリのインポートが完了しました")

✅ ライブラリのインポートが完了しました


In [2]:
# AWS設定
aws_region = os.getenv("AWS_DEFAULT_REGION", "ap-northeast-1")
model_id = "anthropic.claude-3-haiku-20240307-v1:0"
temperature = 0.1

# モデルの初期化
bedrock_model = BedrockModel(
    model_id=model_id, region_name=aws_region, temperature=temperature
)

print(f"✅ モデルを初期化しました: {model_id}")
print(f"リージョン: {aws_region}")
print(f"温度: {temperature}")

2025-06-28 12:31:21,164 - INFO - Found credentials in environment variables.


✅ モデルを初期化しました: anthropic.claude-3-haiku-20240307-v1:0
リージョン: ap-northeast-1
温度: 0.1


## 2. 基本的な使用例（シンプルな質問応答）

In [3]:
# シンプルなエージェントの作成
simple_agent = Agent(model=bedrock_model)

# 質問を投げかける
question = "Pythonの主な特徴を3つ教えてください。"
print(f"質問: {question}\n")

# 応答を取得
response = simple_agent(question)
print(f"応答:\n{response}")

2025-06-28 12:31:26,820 - INFO - Creating Strands MetricsClient


質問: Pythonの主な特徴を3つ教えてください。

Pythonの主な特徴は以下の3つが挙げられます:

1. 高い可読性:
   - Pythonのコードは英語に近い自然な文法を使うため、他の言語に比べて非常に読みやすい。
   - インデントを使ってブロック構造を表現するため、コードの構造が明確になる。

2. 高い生産性:
   - 豊富な標準ライブラリとサードパーティライブラリにより、様々な機能を簡単に実装できる。
   - 動的型付けにより、型宣言の手間が省ける。

3. 汎用性:
   - Webアプリケーション、データ解析、機械学習、自動化など、幅広い分野で使用できる。
   - 簡単なスクリプトから大規模なシステムまで、さまざまなスケールに対応できる。

これらの特徴により、Pythonは初心者にも扱いやすく、経験者にも高い生産性を発揮できる言語として人気を集めています。応答:
Pythonの主な特徴は以下の3つが挙げられます:

1. 高い可読性:
   - Pythonのコードは英語に近い自然な文法を使うため、他の言語に比べて非常に読みやすい。
   - インデントを使ってブロック構造を表現するため、コードの構造が明確になる。

2. 高い生産性:
   - 豊富な標準ライブラリとサードパーティライブラリにより、様々な機能を簡単に実装できる。
   - 動的型付けにより、型宣言の手間が省ける。

3. 汎用性:
   - Webアプリケーション、データ解析、機械学習、自動化など、幅広い分野で使用できる。
   - 簡単なスクリプトから大規模なシステムまで、さまざまなスケールに対応できる。

これらの特徴により、Pythonは初心者にも扱いやすく、経験者にも高い生産性を発揮できる言語として人気を集めています。



In [ ]:
# 複数の質問を連続して処理
questions = [
    "機械学習とは何ですか？",
    "深層学習と機械学習の違いは？",
    "ニューラルネットワークの基本的な仕組みを説明してください。",
]

for i, q in enumerate(questions, 1):
    print(f"\n{'=' * 50}")
    print(f"質問{i}: {q}")
    print(f"{'=' * 50}\n")

    response = simple_agent(q)
    print(f"応答:\n{response}")

## 3. MCPツール統合

Model Context Protocol (MCP)を使用して、外部ツールと統合します。

In [4]:
# MCPクライアントの作成（AWS Documentation Server）
mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

print("✅ MCPクライアントを作成しました")

✅ MCPクライアントを作成しました


In [19]:
import time
import json

# MCPツールを使用したエージェントの作成
with mcp_client:
    # 利用可能なツールを取得
    tools = mcp_client.list_tools_sync()
    print(f"\n📊 利用可能なツール数: {len(tools)}\n")

    # ツール情報の表示
    for i, tool in enumerate(tools[:5]):  # 最初の5つのツールを表示
        # MCPAgentToolオブジェクトの正しい属性アクセス
        if hasattr(tool, "mcp_tool"):
            tool_name = tool.mcp_tool.name
            tool_desc = tool.mcp_tool.description or "説明なし"
        elif hasattr(tool, "tool_name"):
            # 代替: tool_nameプロパティを使用
            tool_name = tool.tool_name
            tool_desc = "説明なし"
        else:
            tool_name = "unknown"
            tool_desc = "説明なし"

        print(f"  {i + 1}. {tool_name}: {tool_desc[:60]}...")

    # ツール使用を詳細にログするコールバックハンドラー（完全パラメータ表示版）
    def complete_tool_callback_handler(**kwargs):
        """ツール使用を詳細にログ、完全なパラメータを蓄積して一度に表示"""

        # ツールパラメータを蓄積するための辞書
        if not hasattr(complete_tool_callback_handler, "tool_inputs"):
            complete_tool_callback_handler.tool_inputs = {}

        try:
            for event_type, event_data in kwargs.items():
                # contentBlockStartでツール使用開始を検出
                if event_type == "event" and isinstance(event_data, dict):
                    if "contentBlockStart" in event_data:
                        content_block = event_data["contentBlockStart"]["start"]
                        if "toolUse" in content_block:
                            tool_use = content_block["toolUse"]
                            tool_id = tool_use.get("toolUseId", "unknown")

                            print(f"\n🔧 ツール使用開始:")
                            print(f"  📛 ツール名: {tool_use.get('name', 'unknown')}")
                            print(f"  🆔 ツールID: {tool_id}")
                            print(f"  ⏰ 時刻: {time.strftime('%H:%M:%S')}")

                            # ツールIDを初期化
                            complete_tool_callback_handler.tool_inputs[tool_id] = ""

                    # ツールの入力パラメータを蓄積
                    elif "contentBlockDelta" in event_data:
                        delta = event_data["contentBlockDelta"]["delta"]
                        if "toolUse" in delta and "input" in delta["toolUse"]:
                            # ツールIDを特定（最新のもの）
                            current_tool_id = (
                                list(complete_tool_callback_handler.tool_inputs.keys())[
                                    -1
                                ]
                                if complete_tool_callback_handler.tool_inputs
                                else None
                            )

                            if current_tool_id:
                                # 入力テキストを蓄積
                                input_text = delta["toolUse"]["input"]
                                complete_tool_callback_handler.tool_inputs[
                                    current_tool_id
                                ] += input_text

                    # ツール実行完了時に完全なパラメータを表示
                    elif "contentBlockStop" in event_data:
                        print(f"\n✅ ツール実行完了")
                        print(f"  ⏰ 完了時刻: {time.strftime('%H:%M:%S')}")

                        # 完全なパラメータを表示
                        if complete_tool_callback_handler.tool_inputs:
                            latest_tool_id = list(
                                complete_tool_callback_handler.tool_inputs.keys()
                            )[-1]
                            complete_input = complete_tool_callback_handler.tool_inputs[
                                latest_tool_id
                            ]

                            print(f"\n⚙️ 完全なツール入力パラメータ:")
                            try:
                                # JSONとしてパース試行
                                parsed_input = json.loads(complete_input)
                                print(
                                    f"  📄 入力: {json.dumps(parsed_input, ensure_ascii=False, indent=4)}"
                                )
                            except json.JSONDecodeError:
                                # JSONでない場合はそのまま表示
                                print(f"  📄 入力: {complete_input}")

                            # 処理済みなので削除
                            del complete_tool_callback_handler.tool_inputs[
                                latest_tool_id
                            ]

                    # メッセージ完了を検出
                    elif "messageStop" in event_data:
                        stop_reason = event_data["messageStop"].get(
                            "stopReason", "unknown"
                        )
                        print(f"\n🏁 メッセージ完了: {stop_reason}")

                    # 使用量情報を表示
                    elif "metadata" in event_data:
                        metadata = event_data["metadata"]
                        if "usage" in metadata:
                            usage = metadata["usage"]
                            print(f"\n📊 使用量情報:")
                            print(
                                f"  📥 入力トークン: {usage.get('inputTokens', 'N/A')}"
                            )
                            print(
                                f"  📤 出力トークン: {usage.get('outputTokens', 'N/A')}"
                            )
                            print(
                                f"  📈 合計トークン: {usage.get('totalTokens', 'N/A')}"
                            )
                        if "metrics" in metadata:
                            metrics = metadata["metrics"]
                            print(
                                f"  ⏱️ レイテンシ: {metrics.get('latencyMs', 'N/A')}ms"
                            )

                # データストリームは簡潔に表示
                elif event_type == "data":
                    data = event_data
                    if len(data.strip()) > 0 and len(data) < 30:
                        print(f"💬 データ: {repr(data)}")

        except Exception as e:
            print(f"❌ コールバックエラー: {e}")
            import traceback

            traceback.print_exc()

    # エージェントの作成（完全パラメータログ付き）
    mcp_agent = Agent(
        model=bedrock_model,
        tools=tools,
        callback_handler=complete_tool_callback_handler,
    )

    # ツール使用を促す具体的な質問
    aws_question = "AWS Lambda関数のベストプラクティスについて最新のAWSドキュメントから詳細情報を取得してください。"
    print(f"\n\n🤖 質問: {aws_question}\n")
    print("=" * 60)
    print("🚀 応答生成開始...")
    print("=" * 60)

    response = mcp_agent(aws_question)

    print("\n" + "=" * 60)
    print("📋 最終応答:")
    print("=" * 60)
    print(f"\n{response}")


📊 利用可能なツール数: 3

  1. read_documentation: Fetch and convert an AWS documentation page to markdown form...
  2. search_documentation: Search AWS documentation using the official AWS Documentatio...
  3. recommend: Get content recommendations for an AWS documentation page.

...


🤖 質問: AWS Lambda関数のベストプラクティスについて最新のAWSドキュメントから詳細情報を取得してください。

🚀 応答生成開始...

🔧 ツール使用開始:
  📛 ツール名: read_documentation
  🆔 ツールID: tooluse_5SvA4f3pTRaTuC_rRwteIg
  ⏰ 時刻: 13:01:48

✅ ツール実行完了
  ⏰ 完了時刻: 13:01:48

⚙️ 完全なツール入力パラメータ:
  📄 入力: {
    "url": "https://docs.aws.amazon.com/lambda/latest/dg/best-practices.html",
    "max_length": 30000
}

🏁 メッセージ完了: tool_use

📊 使用量情報:
  📥 入力トークン: 1718
  📤 出力トークン: 63
  📈 合計トークン: 1781
  ⏱️ レイテンシ: 1176ms
💬 データ: '\n\nThe'
💬 データ: ' key'
💬 データ: ' AWS'
💬 データ: ' Lambda best'
💬 データ: ' practices covere'
💬 データ: 'd in the'
💬 データ: ' documentation'
💬 データ: ' include'
💬 データ: ':\n\n1'
💬 データ: '.'
💬 データ: ' **Function'
💬 データ: ' Code'
💬 データ: '**'
💬 データ: ':'
💬 データ: '\n   -'
💬 データ: ' Leverage'
💬 データ: ' 

## 4. ストリーミング応答

リアルタイムで応答を受け取る方法を示します。

In [16]:
import time
from IPython.display import display, HTML, clear_output


# ストリーミング用のコールバックハンドラー
class StreamingHandler:
    def __init__(self):
        self.content = ""
        self.status = ""

    def __call__(self, **kwargs):
        if "data" in kwargs:
            # 応答データの追加
            self.content += kwargs["data"]
            self.update_display()
        elif "current_tool_use" in kwargs:
            # ツール使用状況
            tool_info = kwargs["current_tool_use"]
            tool_name = tool_info.get("name", "unknown")
            self.status = f"🔧 ツール使用中: {tool_name}"
            self.update_display()
        elif "reasoning" in kwargs:
            # 思考プロセス
            self.status = "🤔 思考中..."
            self.update_display()

    def update_display(self):
        clear_output(wait=True)
        html_content = f"""
        <div style="font-family: monospace; padding: 10px; background-color: #f0f0f0; border-radius: 5px;">
            <div style="color: #666; margin-bottom: 10px;">{self.status}</div>
            <div style="white-space: pre-wrap;">{self.content}</div>
        </div>
        """
        display(HTML(html_content))


# ストリーミングハンドラーのインスタンス化
handler = StreamingHandler()

# ストリーミング対応エージェントの作成
streaming_agent = Agent(model=bedrock_model, callback_handler=handler)

# ストリーミング応答のテスト
streaming_question = (
    "プログラミング言語の選び方について、初心者向けにアドバイスをください。"
)
print(f"質問: {streaming_question}\n")
print("ストリーミング応答:")

response = streaming_agent(streaming_question)

## 5. デバッグとエラーハンドリング

エージェントの動作をデバッグし、エラーを適切に処理する方法を示します。

In [ ]:
# デバッグ用の詳細なコールバックハンドラー
class DebugHandler:
    def __init__(self):
        self.logs = []

    def __call__(self, **kwargs):
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")

        if "current_tool_use" in kwargs:
            tool_info = kwargs["current_tool_use"]
            self.logs.append(
                {"timestamp": timestamp, "type": "tool_use", "data": tool_info}
            )
            print(f"[{timestamp}] 🔧 ツール使用: {tool_info.get('name', 'unknown')}")

        elif "reasoning" in kwargs:
            self.logs.append(
                {"timestamp": timestamp, "type": "reasoning_start", "data": "思考開始"}
            )
            print(f"[{timestamp}] 🤔 思考開始")

        elif "reasoningText" in kwargs:
            reasoning = kwargs["reasoningText"]
            self.logs.append(
                {"timestamp": timestamp, "type": "reasoning_text", "data": reasoning}
            )
            preview = reasoning[:100] + "..." if len(reasoning) > 100 else reasoning
            print(f"[{timestamp}] 💭 思考内容: {preview}")

        elif "data" in kwargs:
            self.logs.append(
                {
                    "timestamp": timestamp,
                    "type": "response_chunk",
                    "data": kwargs["data"],
                }
            )
            # 応答チャンクは表示しない（多すぎるため）

        elif "complete" in kwargs and kwargs["complete"]:
            self.logs.append(
                {"timestamp": timestamp, "type": "complete", "data": "処理完了"}
            )
            print(f"[{timestamp}] ✅ 処理完了")

    def get_summary(self):
        """処理の要約を取得"""
        summary = {
            "total_logs": len(self.logs),
            "tool_uses": sum(1 for log in self.logs if log["type"] == "tool_use"),
            "reasoning_steps": sum(
                1 for log in self.logs if log["type"] == "reasoning_text"
            ),
        }
        return summary


# デバッグハンドラーのインスタンス化
debug_handler = DebugHandler()

# デバッグ対応エージェントの作成
debug_agent = Agent(model=bedrock_model, callback_handler=debug_handler)

# デバッグ実行
debug_question = "Pythonでファイルを読み書きする方法を教えてください。"
print(f"質問: {debug_question}\n")
print("=" * 50)
print("デバッグログ:")
print("=" * 50)

response = debug_agent(debug_question)

print("\n" + "=" * 50)
print("処理の要約:")
print("=" * 50)
summary = debug_handler.get_summary()
for key, value in summary.items():
    print(f"{key}: {value}")

print("\n" + "=" * 50)
print("最終応答:")
print("=" * 50)
print(response)

In [ ]:
# エラーハンドリングの例
def safe_agent_call(agent, message):
    """エラーハンドリング付きのエージェント呼び出し"""
    try:
        response = agent(message)
        return {"success": True, "response": response, "error": None}
    except Exception as e:
        logger.error(f"エージェント実行中にエラーが発生しました: {str(e)}")
        return {"success": False, "response": None, "error": str(e)}


# エラーハンドリングのテスト
test_messages = [
    "正常な質問: Pythonのリスト内包表記について説明してください。",
    "",  # 空の入力
    "非常に長い質問: " + "a" * 10000,  # 極端に長い入力
]

for msg in test_messages:
    print("\n" + "=" * 50)
    print(f"テスト入力: {msg[:50]}..." if len(msg) > 50 else f"テスト入力: {msg}")
    print("=" * 50)

    result = safe_agent_call(simple_agent, msg)

    if result["success"]:
        print("✅ 成功")
        print(
            f"応答: {result['response'][:200]}..."
            if len(result["response"]) > 200
            else f"応答: {result['response']}"
        )
    else:
        print("❌ エラー")
        print(f"エラー内容: {result['error']}")

## まとめ

このノートブックでは、Strands Agentsの以下の機能を学習しました：

1. **基本的な使用方法**: シンプルな質問応答エージェントの作成
2. **MCPツール統合**: 外部ツール（AWS Documentation Server）との連携
3. **ストリーミング応答**: リアルタイムで応答を表示
4. **デバッグとエラーハンドリング**: エージェントの動作を詳細に追跡し、エラーを適切に処理

これらの例を参考に、独自のエージェントアプリケーションを構築してください！

In [ ]:
# リソースのクリーンアップ（必要に応じて実行）
print("🧹 リソースのクリーンアップ完了")